In [ ]:
import ipyparallel
import hddm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

print(hddm.__version__)

In [4]:
data = hddm.load_csv('/data/data.csv')

In [5]:
data = hddm.utils.flip_errors(data)

In [6]:
data = data.drop(data[data.stim == '1Vne1A'].index)
data['movie_valence'] = data.stim.str[:1]
data['movie_arousal'] = data.stim.str[-2:-1]
di = {'group1': "1V1A", 'group2': "1V0A", 'group3': "0V1A", 'group4': "0V0A"}
data['group_code'] = data.group
data = data.replace({"group": di})
data['mood_valence'] = data.group.str[:1]
data['mood_arousal'] = data.group.str[-2:-1]
data.head(10)

,Unnamed: 0,subj_idx,stim,rt,response,group,arousal,valence,participant,movie_valence,movie_arousal,group_code,mood_valence,mood_arousal
0,1,0,0V0A,3.570900,1,0V0A,3,3,96503,0,0,group4,0,0
1,2,0,0V0A,2.457900,1,0V0A,3,3,96503,0,0,group4,0,0
2,3,0,0V0A,1.630200,1,0V0A,3,3,96503,0,0,group4,0,0
3,4,0,0V0A,1.507400,1,0V0A,3,3,96503,0,0,group4,0,0
4,5,0,0V0A,3.756100,1,0V0A,3,3,96503,0,0,group4,0,0
5,6,0,0V0A,4.819273,1,0V0A,3,3,96503,0,0,group4,0,0
6,7,0,0V0A,1.315400,1,0V0A,3,3,96503,0,0,group4,0,0
7,8,0,0V0A,1.541300,1,0V0A,3,3,96503,0,0,group4,0,0
8,9,0,0V0A,-3.250300,0,0V0A,3,3,96503,0,0,group4,0,0
9,10,0,0V0A,1.773600,1,0V0A,3,3,96503,0,0,group4,0,0


In [7]:
m_full_model = hddm.HDDMRegressor(data, "v ~ movie_valence:mood_valence"
                                  + "+ movie_arousal:mood_arousal", 
                                  depends_on={'v': ['mood_valence', 'mood_arousal']}, 
                                  p_outlier=0.05)

Adding these covariates:
['v_Intercept', 'v_mood_valence[T.1]', 'v_movie_valence[T.1]:mood_valence[0]', 'v_movie_valence[T.1]:mood_valence[1]', 'v_mood_arousal[T.1]', 'v_movie_arousal[T.1]:mood_arousal[0]', 'v_movie_arousal[T.1]:mood_arousal[1]']


In [ ]:
m_FULL = hddm.HDDMRegressor(data, ["v ~ movie_valence:mood_valence + movie_arousal:mood_arousal",
                                   "a ~ movie_valence:mood_valence + movie_arousal:mood_arousal",
                                   "z ~ movie_valence:mood_valence + movie_arousal:mood_arousal"],
                            depends_on={'v': ['mood_valence', 'mood_arousal']
                                         'a': ['mood_valence', 'mood_arousal']
                                         'z': ['mood_valence', 'mood_arousal']},
                            include=('v', 'a', 't', 'z'),
                            p_outlier=0.05)

In [ ]:
m_full_model.sample(5000, burn=200)

 [--------         21%                  ] 1081 of 5000 complete in 545.7 sec

In [ ]:
m_FULL.sample(5000, burn=200)

In [ ]:
for row in m_full_model.nodes_db.index: 
    print(row, end = " ")

In [ ]:
v_intercept, v_mood_valence, v_movie_valence_mood_valence_0, \
v_movie_valence_mood_valence_1, v_mood_arousal, v_movie_arousal_mood_arousal_0, \
v_movie_arousal_mood_arousal_1  = m_full_model.nodes_db.loc[["v_Intercept",
                                              "v_mood_valence[T.1]",
                                              "v_movie_valence[T.1]:mood_valence[0]",
                                              "v_movie_valence[T.1]:mood_valence[1]",
                                              "v_mood_arousal[T.1]",
                                              "v_movie_arousal[T.1]:mood_arousal[0]",
                                              "v_movie_arousal[T.1]:mood_arousal[1]"], 'node']


In [ ]:
v_Intercept, v_mood_valence, v_movie_valence_mood_valence_0, v_movie_valence_mood_valence_1,\
v_mood_arousal, v_movie_arousal_mood_arousal_0, v_movie_arousal_mood_arousal_1,\
a_Intercept, a_mood_valence, a_movie_valence_mood_valence_0, a_movie_valence_mood_valence_1,\
a_mood_arousal, a_movie_arousal_mood_arousal_0, a_movie_arousal_mood_arousal_1,\
t_Intercept, t_mood_arousal, t_movie_arousal_mood_arousal_0, t_movie_arousal_mood_valence_1 = \
                                              m_FULL.nodes_db.loc[["v_Intercept",
                                              "v_mood_valence[T.1]",
                                              "v_movie_valence[T.1]:mood_valence[0]",
                                              "v_movie_valence[T.1]:mood_valence[1]",
                                              "v_mood_arousal[T.1]",
                                              "v_movie_arousal[T.1]:mood_arousal[0]",
                                              "v_movie_arousal[T.1]:mood_arousal[1]",
                                                                  "a_Intercept",
                                              "a_mood_valence[T.1]",
                                              "a_movie_valence[T.1]:mood_valence[0]",
                                              "a_movie_valence[T.1]:mood_valence[1]",
                                               "a_mood_arousal[T.1]",
                                              "a_movie_arousal[T.1]:mood_arousal[0]",
                                              "a_movie_arousal[T.1]:mood_arousal[1]",
                                                                  "t_Intercept",
                                              "t_mood_valence[T.1]",
                                              "t_movie_valence[T.1]:mood_valence[0]",
                                              "t_movie_valence[T.1]:mood_valence[1]",
                                                                  "t_mood_arousal[T.1]",
                                              "t_movie_arousal[T.1]:mood_arousal[0]",
                                              "t_movie_arousal[T.1]:mood_arousal[1]"], 'node']

In [ ]:
hddm.analyze.plot_posterior_nodes([v_movie_valence_mood_valence_0,
                                   v_movie_valence_mood_valence_1,
                                   v_movie_arousal_mood_arousal_0,
                                   v_movie_arousal_mood_arousal_1])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Group mean posteriors of within-subject drift-rate effects.')

In [ ]:
print("P = ", (v_movie_valence_mood_valence_1.trace() > v_movie_valence_mood_valence_0.trace()).mean())
print("P = ", (v_movie_valence_mood_valence_1.trace() > 0).mean())
print("P = ", (v_movie_valence_mood_valence_0.trace() < 0).mean())

In [ ]:
print("test1 model DIC: %f" % m_full_model.dic)
print("test3 model DIC: %f" % m_FULL.dic)

In [ ]:
m.plot_posteriors(['a', 't', 'v'])
v_0V0A, v_0V1A, v_1V0A, v_1V1A, v_1Vne1A = m.nodes_db.node[['v(0V0A)', 'v(0V1A)', 'v(1V0A)', 'v(1V1A)', 'v(1Vne1A)']]
hddm.analyze.plot_posterior_nodes([v_0V0A, v_0V1A, v_1V0A, v_1V1A, v_1Vne1A])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Posterior of drift-rate group means')

In [ ]:
m_full_model.plot_posteriors(['a', 't', 'v'])

In [ ]:
stats = m_full_model.gen_stats()

v_stats = stats[stats.index.str.match('v_subj')]


In [ ]:
models = []
for i in range(5):
    model = hddm.HDDMRegressor(data, "v ~ movie_valence:mood_valence"
                                  + "+ movie_arousal:mood_arousal", 
                                  depends_on={'v': ['mood_valence', 'mood_arousal']}, 
                                  p_outlier=0.05)
    model.find_starting_values()
    model.sample(5000, burn=100)
    models.append(model)
    
hddm.analyze.gelman_rubin(models)